# My Toronto Neigborhoods Segmenting and Clustering Exercise

## Note: all three parts of the exercise are included in this notebook:
<b><ul style="list-style-type:none"><li>Part One: Web scraping the postal codes into a dataframe</li><li>Part Two: Latitude & Longitude for the dataframe</li><li>Part Three: Segmenting and Clustering the Neighborhoods in Toronto</li></ol></b>
## Part One: Web scraping the postal codes into a dataframe

In [1]:
pip install beautifulsoup4

     |████████████████████████████████| 102kB 17.5MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

     |████████████████████████████████| 5.8MB 21.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


## Here is the python code to scrape the postal codes of Toronto from Wikipedia

The postal codes of Toronto neighborhoods starting with the letter M are read from the English Wikipedia and added into a dataframe. The method of reading pages from the interweb is called web scraping. The html structure and content of the wikitable are scraped with BeautifulSoup and only the rows from the wikitable are selected. All html tags are discarded and the text from inside the table cell is moved into the dataframe. Here are the rules for processing the rows from Wikipedia page:<ul><li>One borough may be divided into several postal code areas.</li><li> There may be several postal codes for neighborhoods in the same borough. When there are many neighborhoods in the same borough under same postal code, all those neighborhoods are combined and listed on one row, separated with comma.</li><li> When the neighborhood is missing, the borough is copied into the neighborhood column.</li><li> Postal codes that are not assigned to any borough or neighborhood are not included in the dataframe.</li></ul> The resulting dataframe has 103 postal codes.

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
re=requests.get(url)
soup = BeautifulSoup(re.text, 'html.parser')

# scraping the sortable wikitable from the website, reading rows from the table (tr tag) into a list
tbl = soup.find('table', attrs={'class':'wikitable sortable'})
rows=tbl.find_all('tr')

# creating new dataframe with three columns
columns=['PostalCode','Borough','Neighborhood']
df=pd.DataFrame(columns=columns)

# reading thru the scraped postalcodes and adding the details into dataframe
prev_postalcode=''
prev_borough='' 
hoods=''
values=''
for i in range(1,len(rows)):
    tds=rows[i].find_all('td')
    # when the borough not assigned
    if (tds[1].text=='Not assigned'):
        # skip the current row and add the prev info into dataframe
        if (prev_borough!=''):
            values=[prev_postalcode, prev_borough, hoods]
            df=df.append(pd.Series(values, index=columns), ignore_index=True)
        prev_postalcode=''
        prev_borough=''
        hoods=''
        continue
    # when the hood is not assigned
    hood2=tds[2].text.replace('\n','').rstrip()
    if (hood2=='Not assigned'):
        if (prev_postalcode!=''):
            values=[prev_postalcode, prev_borough, hoods]
            df=df.append(pd.Series(values, index=columns), ignore_index=True)
        prev_postalcode=tds[0].text
        prev_borough=tds[1].text
        hoods=tds[1].text
        continue
    # when the postalcode and the borough is the same as on previous row
    if (tds[0].text==prev_postalcode and tds[1].text==prev_borough):
        hoods=hoods+', '+tds[2].text.replace('\n','')
        continue
    else:
        # the regular case where postalcode changes and a row is written
        if (prev_postalcode!='' and prev_borough!='' and hoods!=''):
            values=[prev_postalcode, prev_borough, hoods]
            df=df.append(pd.Series(values, index=columns), ignore_index=True)
        prev_postalcode=tds[0].text
        prev_borough=tds[1].text
        hoods=tds[2].text.replace('\n','') 
        
# writing the last row after the loop 
if (prev_postalcode!=''):
    values=[prev_postalcode, prev_borough, hoods]
    df=df.append(pd.Series(values, index=columns), ignore_index=True)

### This is how the resulting dataframe looks like

In [6]:
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### The number of rows and columns on dataframe

In [7]:
df.shape

(103, 3)

## Part Two: Latitude & Longitude for the dataframe

I am using the <a href>https://cocl.us/Geospatial_data</a> csv-file for the latitude and longitude of Toronto neighborhoods. The contents of the file are: postal code, latitude, longitude. In the previous stage of the Toronto neighbos exercise the Toronto postal codes were brought into a dataframe with only one row for each postal code.

In [8]:
import pandas as pd

# reading the csv-file into a dataframe
geo_df = pd.read_csv('https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')

# A look into the geo data
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
# The number of rows in the geospatial coordinates df
geo_df.shape

(103, 3)

In [10]:
# copying dataframe from previous stage to another name
toronto_df=df

# adding two new columnd with default value 0 to toronto_df
toronto_df['Latitude'] = 0
toronto_df['Longitude'] = 0

# reading values for longitude and latitude for each postal code
for index, row in geo_df.iterrows():
    # setting longitudes for toronto_df
    toronto_df.loc[toronto_df['PostalCode']==row['Postal Code'], ['Longitude']] = row['Longitude']
    # setting latitudes for toronto_df
    toronto_df.loc[toronto_df['PostalCode']==row['Postal Code'], ['Latitude']]= row['Latitude']

# Taking a look at the final resultset
toronto_df.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## Part Three: Segmenting and Clustering the Neighborhoods in Toronto

### Importing libraries required for the exercise.

In [11]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # run this if necessary
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         237 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

First, we need to locate the coordinates for Toronto

In [12]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

### For the segmenting and analysis part, I am concentrating on <b>Downtown Toronto</b> and running through the exercises previously conducted for the New York Neighborhoods dataset.
There are 18 rows in the Downtown Toronto dataset.

In [24]:
downtown_data = toronto_df[toronto_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)

print('{} venues were returned by Foursquare.'.format(downtown_data.shape[0]))
downtown_data.head(18)

18 venues were returned by Foursquare.


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564
6,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
7,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
8,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


## Geo coordinates for Downtown Toronto

In [16]:
address = 'Downtown Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Downtown Toronto are 43.655115, -79.380219.


## Creating folium map of Downtown Toronto with the postal areas marked with blue circles.

In [17]:
# create map using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

## My Foursquare credentials

In [18]:
CLIENT_ID = 'PWCQCVWVSQOBTG5FXVEXLGD3BTEHWT551EC0E4HXK3N1AE1N' # your Foursquare ID
CLIENT_SECRET = 'V1UUUG3B44KJG2HBQN0TRIXASGMSIECXLUEMYWEDDJIIX1JV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: PWCQCVWVSQOBTG5FXVEXLGD3BTEHWT551EC0E4HXK3N1AE1N
CLIENT_SECRET:V1UUUG3B44KJG2HBQN0TRIXASGMSIECXLUEMYWEDDJIIX1JV


### Let's look at the M4X postal code area Downtown Toronto, namely Cabbagetown and St. James Town

In [71]:
neighborhood_latitude = downtown_data.loc[15, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[15, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[15, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Cabbagetown, St. James Town are 43.667967, -79.3676753.


Now, let's search for top venues in the Cabbagetown/St. James Town area

In [72]:
search_query='venue'
radius=500
LIMIT=50

url = 'https://api.foursquare.com/v2/venues/search?client_id=PWCQCVWVSQOBTG5FXVEXLGD3BTEHWT551EC0E4HXK3N1AE1N&client_secret=V1UUUG3B44KJG2HBQN0TRIXASGMSIECXLUEMYWEDDJIIX1JV&ll=43.667967,-79.3676753&v=20180604&search_query=venue&radius=500&limit=50'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/search?client_id=PWCQCVWVSQOBTG5FXVEXLGD3BTEHWT551EC0E4HXK3N1AE1N&client_secret=V1UUUG3B44KJG2HBQN0TRIXASGMSIECXLUEMYWEDDJIIX1JV&ll=43.667967,-79.3676753&v=20180604&search_query=venue&radius=500&limit=50'

Sending the GET request to see the results.

In [80]:
results = requests.get(url).json()

Moving results into pandas dataframe

In [81]:
# picking venues from the result set
venues = results['response']['venues']
    
venues_df = json_normalize(venues) # flatten JSON

In [82]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in venues_df.columns if col.startswith('location.')] + ['id']
dataframe_filtered = venues_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

venues_df=dataframe_filtered

print('Dataframe filtered!')
venues_df.head()

Dataframe filtered!


,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,address,crossStreet,neighborhood,id
0,51 Winchester St,Event Space,43.665830,-79.368198,"[{'label': 'display', 'lat': 43.66583, 'lng': ...",241,M6H,CA,Toronto,ON,Canada,"[Toronto ON M6H, Canada]",NaN,NaN,NaN,5cd9d146364d97002c480a16
1,F'Amelia,Italian Restaurant,43.667536,-79.368613,"[{'label': 'display', 'lat': 43.66753590663226...",89,M4X 1E1,CA,Toronto,ON,Canada,"[12 Amelia St (Parliament St), Toronto ON M4X ...",12 Amelia St,Parliament St,NaN,4e4e7aa06365e1419d021044
2,Esso,Gas Station,43.666995,-79.368933,"[{'label': 'display', 'lat': 43.66699457454887...",148,M4X 1P7,CA,Toronto,ON,Canada,"[581 Parliament St (at Amelia St), Toronto ON ...",581 Parliament St,at Amelia St,NaN,4bd0bf1cb221c9b63e52d4d0
3,LCBO,Liquor Store,43.665586,-79.368531,"[{'label': 'display', 'lat': 43.66558559308836...",273,M4X 1P4,CA,Toronto,ON,Canada,"[512 Parliament St. (at Winchester St.), Toron...",512 Parliament St.,at Winchester St.,NaN,4b7c8c91f964a520569a2fe3
4,Menagerie Pet Shop,Pet Store,43.666445,-79.368774,"[{'label': 'display', 'lat': 43.6664445877461,...",191,M4X 1P7,CA,Toronto,ON,Canada,[549 Parliament St. (btwn Carlton & Wellesley)...,549 Parliament St.,btwn Carlton & Wellesley,NaN,4b1985bff964a520cbde23e3


In [83]:
print('{} venues were returned by Foursquare.'.format(dataframe_filtered.shape[0]))

50 venues were returned by Foursquare.


#### How many unique categories on M4X postal area

In [92]:
print('There are {} unique categories'.format(len(venues_df['categories'].unique())))

There are 39 unique categories


Let's take a look at one of the venues at the area, The Phoenix Concert Theatre (venue id 4ad4c062f964a520cbf720e3

In [94]:
# Foursquare query
venue_id='4ad4c062f964a520cbf720e3'

url = 'https://api.foursquare.com/v2/venues/4ad4c062f964a520cbf720e3?client_id=PWCQCVWVSQOBTG5FXVEXLGD3BTEHWT551EC0E4HXK3N1AE1N&client_secret=V1UUUG3B44KJG2HBQN0TRIXASGMSIECXLUEMYWEDDJIIX1JV&ll=43.667967,-79.3676753&v=20180604&search_query=venue&radius=500&limit=50'.format(venue_id, CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/4ad4c062f964a520cbf720e3?client_id=PWCQCVWVSQOBTG5FXVEXLGD3BTEHWT551EC0E4HXK3N1AE1N&client_secret=V1UUUG3B44KJG2HBQN0TRIXASGMSIECXLUEMYWEDDJIIX1JV&ll=43.667967,-79.3676753&v=20180604&search_query=venue&radius=500&limit=50'

In [152]:
result = requests.get(url).json()
#print(result['response']['venue'].keys())
result['response']['venue']

df3 = json_normalize(result)

filtered_columns = ['response.venue.name', 'response.venue.location.address','response.venue.location.postalCode',\
                    'response.venue.location.city','response.venue.location.cc','response.venue.rating',\
                    'response.venue.likes.count','response.venue.photos.count']
df3 = df3.loc[:, filtered_columns]

df3.rename(columns={'response.venue.name':'Venue Name','response.venue.location.address':'Address','response.venue.location.postalCode':'Postal Code',\
                    'response.venue.location.city':'City','response.venue.location.cc':'Country','response.venue.rating':'Rating',\
                    'response.venue.likes.count':'Likes','response.venue.photos.count':'Photos'},inplace=True)
df3

,Venue Name,Address,Postal Code,City,Country,Rating,Likes,Photos
0,Phoenix Concert Theatre,410 Sherbourne St.,M4X 1K2,Toronto,CA,7.6,162,392


## 2. Exploring Neighborhoods in Downtown Toronto

### Getting the nearby venues for latitudes, longitudes on postal areas in the dataset

In [84]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)   # printing the downtown toronto neighborhood names
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [85]:

downtown_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

print(downtown_venues.shape)
downtown_venues.head()

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley
(763, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


### Counting the number of venues on each neighborhood/postal area (max 50)

In [86]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,50,50,50,50,50,50
"Chinatown, Grange Park, Kensington Market",50,50,50,50,50,50
Christie,17,17,17,17,17,17
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50


In [87]:
print('There are {} unique categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 178 unique categories.


## 3. Analyzing Neighborhoods

In [153]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column (which didn't seem to work)
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [154]:
downtown_onehot.shape

(763, 178)

In [155]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.00,0.00,0.00,0.00,0.04,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.040000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.02,0.040000,0.00,0.00,0.0,0.02,0.02,0.000000,0.060000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.080000,0.000000,0.000000,0.02,0.000000,0.00,0.02,0.000000,0.00,0.00,0.00,0.020000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.000000,0.00,0.00,0.00,0.040000,0.00,0.000000,0.02,0.000000,0.00,0.000000,0.02,0.000000,0.000000,0.02,0.000000,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.000000,0.000000,0.020000,0.02,0.000000,0.00,0.00,0.00,0.000000,0.02,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.000000,0.020000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.040000,0.000000,0.02,0.00,0.00,0.00,0.000000,0.00,0.02,0.000000,0.00,0.00,0.00,0.00,0.020000,0.00,0.000000,0.02,0.00,0.00,0.00,0.02,0.00,0.000000,0.00,0.02,0.00,0.00,0.06,0.00,0.020000,0.00,0.00,0.000000,0.00,0.00,0.020000,0.000000,0.00,0.00,0.0,0.00,0.02,0.000000,0.00,0.00,0.00
1,Berczy Park,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.02,0.00,0.00,0.00,0.000000,0.00,0.000000,0.02,0.040000,0.000000,0.000000,0.00,0.02,0.02,0.040000,0.000000,0.02,0.02,0.000000,0.000000,0.000000,0.00,0.020000,0.00,0.00,0.0,0.00,0.00,0.000000,0.040000,0.00,0.000000,0.04,0.000000,0.00,0.00,0.02,0.06,0.080000,0.000000,0.000000,0.00,0.020000

In [156]:
downtown_grouped.shape

(18, 178)

### Listing top categries by frequence on each postal area /Hood in Downtown Toronto

In [157]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.08
1   Steakhouse  0.06
2         Café  0.06
3    Gastropub  0.04
4  Pizza Place  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.06
2            Beer Bar  0.04
3         Cheese Shop  0.04
4  Seafood Restaurant  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
             venue  freq
0   Airport Lounge  0.14
1  Airport Service  0.14
2      Coffee Shop  0.07
3  Harbor / Marina  0.07
4    Boat or Ferry  0.07


----Cabbagetown, St. James Town----
         venue  freq
0  Coffee Shop  0.07
1   Restaurant  0.05
2       Bakery  0.05
3          Pub  0.05
4         Park  0.05


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.16
1                       Café  0.06
2  Middle Eastern Restaurant  0.04
3             Ice Cream Shop  0.04
4               

In [158]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [159]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Pizza Place,Breakfast Spot,Bar,Asian Restaurant,Hotel,Gastropub
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Steakhouse,Cheese Shop,Seafood Restaurant,Café,Bakery,Creperie
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Coffee Shop,Harbor / Marina,Boutique,Bar,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport
3,"Cabbagetown, St. James Town",Coffee Shop,Café,Bakery,Park,Italian Restaurant,Pizza Place,Market,Pub,Restaurant,Breakfast Spot
4,Central Bay Street,Coffee Shop,Café,Spa,Italian Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Bubble Tea Shop,Ice Cream Shop,Bar,Hotel


## 4. Clustering Neighborhoods

In [160]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 0, 0, 4, 2, 0, 0, 0], dtype=int32)

In [161]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Café,Mexican Restaurant,Gym / Fitness Center,Theater,Restaurant,Breakfast Spot
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Café,Clothing Store,Fast Food Restaurant,Theater,Ramen Restaurant,Cosmetics Shop,Bookstore,Sandwich Place,Italian Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Gastropub,Restaurant,Café,Hotel,Farmers Market,Italian Restaurant,Japanese Restaurant,Park,Cocktail Bar
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Steakhouse,Cheese Shop,Seafood Restaurant,Café,Bakery,Creperie
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Spa,Italian Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Bubble Tea Shop,Ice Cream Shop,Bar,Hotel


In [163]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.prism(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

## 5. Examining the Clusters

Determining what categories distinguish each cluster is not easy because of my choice of Downtown Toronto. Only the airport district and the outdoors area are clearly separate from the rest of the clusters. Downtown Toronto is full of tourist attractions and services in every part of the area. Yet, ordinary Toronto residents live there. Most interesting is the cluster where coffee shops are the most common venues, was thinking of a coincidence. 

#### Cluster 1, The Airport

In [75]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Bar,Airport,Airport Food Court,Airport Gate,Coffee Shop,Sculpture Garden


#### Cluster 2, Tourism & shopping

In [76]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1,Café,Clothing Store,Beer Bar,Pizza Place,Comic Shop,Shopping Mall,Coffee Shop,Spa,Ramen Restaurant,Plaza
3,Downtown Toronto,1,Cocktail Bar,Beer Bar,Seafood Restaurant,Farmers Market,Clothing Store,Concert Hall,Basketball Stadium,Liquor Store,Bakery,Breakfast Spot
6,Downtown Toronto,1,Steakhouse,Asian Restaurant,Hotel,Coffee Shop,Seafood Restaurant,Smoke Shop,Pizza Place,Speakeasy,Concert Hall,Sushi Restaurant
10,Downtown Toronto,1,Café,Bookstore,Restaurant,Sandwich Place,Bakery,Japanese Restaurant,Bar,Italian Restaurant,College Arts Building,College Gym
11,Downtown Toronto,1,Café,Vietnamese Restaurant,Mexican Restaurant,Caribbean Restaurant,Coffee Shop,Fish Market,Farmers Market,Dessert Shop,Comfort Food Restaurant,Cheese Shop
14,Downtown Toronto,1,Beer Bar,Café,Cocktail Bar,Farmers Market,Bakery,Park,Comfort Food Restaurant,Jazz Club,Seafood Restaurant,Clothing Store


#### Cluster 3, Toronto Residential

In [77]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,2,Grocery Store,Café,Park,Restaurant,Diner,Baby Store,Athletics & Sports,Italian Restaurant,Convenience Store,Coffee Shop
7,Downtown Toronto,2,Hotel,Café,Park,Plaza,Performing Arts Venue,Chinese Restaurant,Bubble Tea Shop,Lake,Salad Place,Bistro
15,Downtown Toronto,2,Italian Restaurant,Café,Restaurant,Bakery,Caribbean Restaurant,Butcher,Pet Store,Coffee Shop,Market,Pub
17,Downtown Toronto,2,Park,Dance Studio,Pub,Ramen Restaurant,Restaurant,Breakfast Spot,Salon / Barbershop,Juice Bar,Bubble Tea Shop,Pizza Place


#### Cluster 4. Coffee & Dining

In [78]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3,Coffee Shop,Park,Breakfast Spot,Gym / Fitness Center,Bakery,Café,Restaurant,Spa,Pub,Dessert Shop
2,Downtown Toronto,3,Coffee Shop,Gastropub,Hotel,Japanese Restaurant,Restaurant,BBQ Joint,Gym,Food Truck,Italian Restaurant,Creperie
4,Downtown Toronto,3,Coffee Shop,Spa,Bubble Tea Shop,Italian Restaurant,Park,Modern European Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Poke Place,Ramen Restaurant
8,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Deli / Bodega,Fried Chicken Joint,Hotel,Gastropub,Pizza Place,Pub,Sandwich Place
9,Downtown Toronto,3,Café,Coffee Shop,Restaurant,Deli / Bodega,Gastropub,Bakery,Gym,Japanese Restaurant,Beer Bar,Hotel
16,Downtown Toronto,3,Café,Coffee Shop,Restaurant,Steakhouse,Asian Restaurant,Pub,Japanese Restaurant,Seafood Restaurant,Bar,Bakery


#### Cluster 5, Outdoors Recreational

In [79]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,4,Park,Playground,Trail,Building,Coffee Shop,Concert Hall,Comic Shop,Comfort Food Restaurant,College Gym,College Arts Building


### This is the end of Toronto Neighborhoods exercise.